In [2]:
import sqlite3
import json
import time
from web3 import Web3
from dotenv import load_dotenv
import os


load_dotenv()

RPC_URL = os.getenv('SEPOLIA_RPC_URL')
CONTRACT_ADDRESS = os.getenv('CONTRACT_ADDRESS')
DB_PATH = 'order_events.db'


ABI_JSON = '''[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"token","type":"address"},{"indexed":true,"internalType":"address","name":"recipient","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"AssetWithdrawn","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"sender","type":"address"},{"indexed":true,"internalType":"address","name":"token","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"},{"indexed":false,"internalType":"uint8","name":"orderType","type":"uint8"},{"indexed":false,"internalType":"uint256","name":"size","type":"uint256"},{"indexed":false,"internalType":"string","name":"side","type":"string"},{"indexed":false,"internalType":"string","name":"marketCode","type":"string"}],"name":"OrderPlaced","type":"event"},{"stateMutability":"payable","type":"fallback"},{"inputs":[],"name":"owner","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"uint8","name":"orderType","type":"uint8"},{"internalType":"uint256","name":"size","type":"uint256"},{"internalType":"string","name":"side","type":"string"},{"internalType":"string","name":"marketCode","type":"string"}],"name":"placeEthOrder","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint8","name":"orderType","type":"uint8"},{"internalType":"uint256","name":"size","type":"uint256"},{"internalType":"string","name":"side","type":"string"},{"internalType":"string","name":"marketCode","type":"string"}],"name":"placeTokenOrder","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"token","type":"address"},{"internalType":"address","name":"recipient","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"withdraw","outputs":[],"stateMutability":"nonpayable","type":"function"},{"stateMutability":"payable","type":"receive"}]'''
ABI = json.loads(ABI_JSON)


def init_db():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS order_placed (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        block_number INTEGER,
        transaction_hash TEXT,
        sender TEXT,
        token TEXT,
        amount TEXT,
        order_type INTEGER,
        size TEXT,
        side TEXT,
        market_code TEXT,
        timestamp INTEGER
    )
    ''')
    

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS asset_withdrawn (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        block_number INTEGER,
        transaction_hash TEXT,
        token TEXT,
        recipient TEXT,
        amount TEXT,
        timestamp INTEGER
    )
    ''')
    

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS last_processed_block (
        id INTEGER PRIMARY KEY,
        block_number INTEGER
    )
    ''')
    
    conn.commit()
    conn.close()
    print("Database initialized successfully.")


def save_last_processed_block(block_number):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    cursor.execute('DELETE FROM last_processed_block')
    cursor.execute('INSERT INTO last_processed_block (id, block_number) VALUES (1, ?)', (block_number,))
    
    conn.commit()
    conn.close()


def get_last_processed_block():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    cursor.execute('SELECT block_number FROM last_processed_block WHERE id = 1')
    result = cursor.fetchone()
    
    conn.close()
    
    if result:
        return result[0]
    return None


def handle_order_placed(event, w3):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    block = w3.eth.get_block(event['blockNumber'])
    timestamp = block.timestamp
    
    cursor.execute('''
    INSERT INTO order_placed (
        block_number, transaction_hash, sender, token, amount, order_type, 
        size, side, market_code, timestamp
    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        event['blockNumber'],
        event['transactionHash'].hex(),
        event['args']['sender'],
        event['args']['token'],
        str(event['args']['amount']),
        event['args']['orderType'],
        str(event['args']['size']),
        event['args']['side'],
        event['args']['marketCode'],
        timestamp
    ))
    
    conn.commit()
    conn.close()
    
    print(f"OrderPlaced event saved: TX {event['transactionHash'].hex()} in block {event['blockNumber']}")


def handle_asset_withdrawn(event, w3):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    
    block = w3.eth.get_block(event['blockNumber'])
    timestamp = block.timestamp
    
    cursor.execute('''
    INSERT INTO asset_withdrawn (
        block_number, transaction_hash, token, recipient, amount, timestamp
    ) VALUES (?, ?, ?, ?, ?, ?)
    ''', (
        event['blockNumber'],
        event['transactionHash'].hex(),
        event['args']['token'],
        event['args']['recipient'],
        str(event['args']['amount']),
        timestamp
    ))
    
    conn.commit()
    conn.close()
    
    print(f"AssetWithdrawn event saved: TX {event['transactionHash'].hex()} in block {event['blockNumber']}")


def process_blocks(contract, w3, from_block, to_block):
    print(f"Processing blocks {from_block} to {to_block}")
    

    order_placed_events = contract.events.OrderPlaced().get_logs(
        from_block=from_block, to_block=to_block
    )
    

    asset_withdrawn_events = contract.events.AssetWithdrawn().get_logs(
        from_block=from_block, to_block=to_block
    )
    

    for event in order_placed_events:
        handle_order_placed(event, w3)
    
    for event in asset_withdrawn_events:
        handle_asset_withdrawn(event, w3)
    
    total_events = len(order_placed_events) + len(asset_withdrawn_events)
    if total_events > 0:
        print(f"Processed {len(order_placed_events)} OrderPlaced and {len(asset_withdrawn_events)} AssetWithdrawn events")
    else:
        print("No new events found")
    

    save_last_processed_block(to_block)
    
    return to_block


def setup_web3():
    w3 = Web3(Web3.HTTPProvider(RPC_URL))
    

    try:
        from web3.middleware import geth_poa_middleware
        w3.middleware_onion.inject(geth_poa_middleware, layer=0)
    except ImportError:
        try:
            from web3.middleware.geth import geth_poa_middleware
            w3.middleware_onion.inject(geth_poa_middleware, layer=0)
        except ImportError:
            print("Could not import geth_poa_middleware, continuing without it")
    
    if not w3.is_connected():
        print("Failed to connect to Ethereum network")
        return None, None
    
    print(f"Connected to Ethereum network. Current block: {w3.eth.block_number}")
    

    contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)
    
    return w3, contract


def initialize_and_process_events():

    init_db()
    

    w3, contract = setup_web3()
    if not w3 or not contract:
        return None
    

    current_block = w3.eth.block_number
    

    last_block = get_last_processed_block()
    if last_block is None:
        start_block = max(1, current_block - 10)
    else:
        start_block = last_block + 1
    

    if start_block <= current_block:
        last_block = process_blocks(contract, w3, start_block, current_block)
    else:
        print(f"No new blocks to process. Last processed: {start_block-1}, Current: {current_block}")
        last_block = current_block
    
    return {
        'web3': w3,
        'contract': contract,
        'last_processed_block': last_block
    }


def start_polling(duration_seconds=60):

    context = initialize_and_process_events()
    if not context:
        return "Failed to initialize"
    
    w3 = context['web3']
    contract = context['contract']
    last_processed_block = context['last_processed_block']
    

    start_time = time.time()
    end_time = start_time + duration_seconds
    

    try:
        while time.time() < end_time:

            print(f"Waiting 15 seconds before next check... (Press Ctrl+C to stop)")
            time.sleep(15)
            

            current_block = w3.eth.block_number
            

            if last_processed_block < current_block:
                last_processed_block = process_blocks(
                    contract, w3, last_processed_block + 1, current_block
                )
            else:
                print(f"No new blocks. Still at block {current_block}")
        
        print(f"Polling completed after {duration_seconds} seconds")
        return "Polling completed"
    
    except KeyboardInterrupt:
        print("Polling stopped by user")
        return "Polling stopped by user"
    except Exception as e:
        print(f"Error during polling: {e}")
        return f"Error: {e}"

def run_event_listener(duration_seconds=60):
    print("Starting Ethereum event listener...")
    return start_polling(duration_seconds)

run_event_listener(duration_seconds=60) 

Starting Ethereum event listener...
Database initialized successfully.
Could not import geth_poa_middleware, continuing without it
Connected to Ethereum network. Current block: 7960347
Processing blocks 7960346 to 7960347
No new events found
Waiting 15 seconds before next check... (Press Ctrl+C to stop)
Processing blocks 7960348 to 7960348
OrderPlaced event saved: TX ec6e479217df06eaf9b6d00711590b78fe77bf93bea0f86d556e4ab33ff2adc7 in block 7960348
Processed 1 OrderPlaced and 0 AssetWithdrawn events
Waiting 15 seconds before next check... (Press Ctrl+C to stop)
Polling stopped by user


'Polling stopped by user'

In [ ]:
import json
from web3 import Web3
from eth_account import Account
import time
from dotenv import load_dotenv
import os


load_dotenv()

RPC_URL = os.getenv('SEPOLIA_RPC_URL')
PRIVATE_KEY = os.getenv('PRIVATE_KEY')
CONTRACT_ADDRESS = os.getenv('CONTRACT_ADDRESS')

ABI = [
    {
        "inputs": [
            {"internalType": "address", "name": "token", "type": "address"},
            {"internalType": "address", "name": "recipient", "type": "address"},
            {"internalType": "uint256", "name": "amount", "type": "uint256"}
        ],
        "name": "withdraw",
        "outputs": [],
        "stateMutability": "nonpayable",
        "type": "function"
    },
    {
        "inputs": [],
        "name": "owner",
        "outputs": [{"internalType": "address", "name": "", "type": "address"}],
        "stateMutability": "view",
        "type": "function"
    }
]

def execute_withdrawal(private_key, token_address, recipient_address, amount_in_ether):


    w3 = Web3(Web3.HTTPProvider(RPC_URL))
    if not w3.is_connected():
        print("Failed to connect to Ethereum network")
        return None
    
    print(f"Connected to network. Current block: {w3.eth.block_number}")
    
    # Setup account from private key
    account = Account.from_key(private_key)
    print(f"Using account: {account.address}")
    
    # Initialize contract
    contract = w3.eth.contract(address=CONTRACT_ADDRESS, abi=ABI)
    
    # Check if user is the owner
    owner = contract.functions.owner().call()
    if owner.lower() != account.address.lower():
        print(f"Error: Only the contract owner can withdraw. Owner is {owner}")
        return None
    
    # Convert amount to Wei
    amount_in_wei = w3.to_wei(amount_in_ether, 'ether')
    

    if token_address.lower() in ['0x0', '0x0000000000000000000000000000000000000000', 'eth']:
        token_address = '0x0000000000000000000000000000000000000000'
        asset_type = 'ETH'
    else:
        asset_type = 'Token'
        
    # Print withdrawal details
    print(f"\nWithdrawal Details:")
    print(f"- Asset: {asset_type}")
    print(f"- Token Address: {token_address}")
    print(f"- Recipient: {recipient_address}")
    print(f"- Amount: {amount_in_ether} ({amount_in_wei} Wei)")
    
    try:
        # Prepare transaction
        tx = contract.functions.withdraw(
            token_address,
            recipient_address,
            amount_in_wei
        ).build_transaction({
            'from': account.address,
            'nonce': w3.eth.get_transaction_count(account.address),
            'gas': 200000,  # Adjust gas as needed
            'gasPrice': w3.eth.gas_price
        })
        
        # Sign transaction
        signed_tx = w3.eth.account.sign_transaction(tx, private_key)
        
        # Send transaction
        print("\nSending transaction...")
        tx_hash = w3.eth.send_raw_transaction(signed_tx.raw_transaction)
        tx_hash_hex = tx_hash.hex()
        print(f"Transaction sent! Hash: {tx_hash_hex}")
        

        print("Waiting for confirmation...")
        tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)        

        if tx_receipt.status == 1:
            print(f"Success! Transaction confirmed in block {tx_receipt.blockNumber}")
            print(f"Gas used: {tx_receipt.gasUsed}")
            return tx_hash_hex
        else:
            print("Transaction failed!")
            return None
        
    except Exception as e:
        print(f"Error executing withdrawal: {e}")
        return None


def execute(private_key, token_address, recipient, amount):

    return execute_withdrawal(
        private_key=private_key,
        token_address=token_address, 
        recipient_address=recipient,
        amount_in_ether=amount
    )




execute(
    private_key=PRIVATE_KEY,
    token_address="0x77f369477A0140b30D359741F8720EE23f03eBD7", 
    recipient="0x790d325cC1f319620aDdd1be1B78924AB1D18311",
    amount=100 )